# 

# Generating synthetic data
- This jupyter note book will run through set up and execution for generating synthetic data

In [ ]:
# Setting up containers
!cd ../umccr-dictionary && make down
!cd ../umccr-dictionary && make pull
!cd ../umccr-dictionary && make up
!cd ../umccr-dictionary && make ps


# Generating synthetic data using umccr-dictionary
- This part of the script assumes that bundled json has been created using:

```bash
!cd ../umccr-dictionary && make compile program=schema_dev
```

- If this has not yet been created, make sure to create it first using the [schema_dev_framework.ipynb notebook](./schema_dev_framework.ipynb)


In [ ]:
# Generating synthetic data using umccr-dictionary
!cd ../umccr-dictionary && make simulate program=schema_dev project=tmp_project max_samples=110

# Running plausible data generation
- Make sure to specify gurl, which reads the SD and mean values from a google sheet and generates a synthetic data set
- Refer to [plausible_values gsheet](https://docs.google.com/spreadsheets/d/1zjDBDvXgb0ydswFBwy47r2c8V1TFnpUj1jcG0xsY7ZI/edit#gid=1771305859)

In [ ]:
# Running plausible data generation
!python3 ../plausible_data_gen.py --path ../output/synthetic_data/schema_dev/tmp_project --gurl 'https://docs.google.com/spreadsheets/d/1zjDBDvXgb0ydswFBwy47r2c8V1TFnpUj1jcG0xsY7ZI/edit#gid=1771305859'

In [4]:
# Copying synthetic data back to synthetic data output
!mkdir -p ../output/synthetic_data
!cp -r ../umccr-dictionary/data/schema_dev ../output/synthetic_data/